## Make Positions

The positions in the GPX file occur approximately every eight hours. To get a smoother animation that allows the progression of the weather to be seen more frequent positions are required. Additionally, to save money we turned the Yellowbrick off when in port. The last position is often out at sea and 

This notebook takes the original fixes, adds some manual positions for when we were in port and then interpolates between the points to get hourly positions. Various interpolation techniques are possible; a simple linear interpolation based on the bearing and distance between fixes will be tried first.

In [9]:
from geographiclib.geodesic import Geodesic
from geopy import distance
import gpxpy
import pandas as pd

In [5]:
gpx_file = 'yblog-1589101179046.gpx'

In [6]:
with open(gpx_file) as fh:
    gpx = gpxpy.parse(fh)
data = gpx.tracks[0].segments[0].points

In [7]:
print(data[0])

[trkpt:55.94507999999999,-4.746780000000001@8.0@2013-09-04 16:19:01+00:00]


In [28]:
df = pd.DataFrame(columns=['lat', 'lon', 'time'])

# Skip the first entry as it was just a test one week before we departed
for point in data[1:]: 
    df = df.append({'lon': point.longitude, 'lat' : point.latitude, 'time' : point.time}, ignore_index=True)

In [30]:
geod = Geodesic.WGS84

dist_dif = []
time_dif = []

# Remember that we're skipping the first entry
for index in range(1, len(data) - 1):
    start = data[index]
    stop = data[index+1]
    distance_2d = geod.Inverse(start.latitude, start.longitude,
                               stop.latitude, stop.longitude)['s12']
    dist_dif.append(distance_2d)
    time_delta = (stop.time - start.time).total_seconds()    
    time_dif.append(time_delta)
# Add the last row
dist_dif.append(0)
time_dif.append(0)
    
df['time_dif'] = time_dif
df['dis_dif'] = dist_dif

In [31]:
print(df[:10])
print(df[-3:])


        lat      lon                       time  time_dif        dis_dif
0  55.94507 -4.74655  2013-09-16 21:54:06+00:00   65160.0   45971.356742
1  55.56232 -5.02120  2013-09-17 16:00:06+00:00   28802.0  100928.014355
2  54.66902 -5.29117  2013-09-18 00:00:08+00:00   28798.0  117976.169012
3  53.64186 -4.84571  2013-09-18 08:00:06+00:00   28800.0   38121.353753
4  53.32146 -4.64268  2013-09-18 16:00:06+00:00   28800.0       5.331001
5  53.32146 -4.64260  2013-09-19 00:00:06+00:00   28800.0       6.397101
6  53.32148 -4.64269  2013-09-19 08:00:06+00:00   28800.0       0.000000
7  53.32148 -4.64269  2013-09-19 16:00:06+00:00   28831.0    2372.313017
8  53.34245 -4.64908  2013-09-20 00:00:37+00:00   28788.0  111154.812503
9  52.48326 -5.49176  2013-09-20 08:00:25+00:00   28813.0  111159.060838
          lat      lon                       time  time_dif       dis_dif
546  58.03371 -6.33969  2014-05-24 00:00:06+00:00   28800.0  19772.891561
547  58.20922 -6.39012  2014-05-24 08:00:06+00:00

In [32]:
hourly = pd.DataFrame(columns=['lat', 'lon', 'time', 'src'])
hourly['time'] = pd.date_range('2013-09-04', '2014-05-25', freq='H')
print(hourly[:10])

   lat  lon                time  src
0  NaN  NaN 2013-09-04 00:00:00  NaN
1  NaN  NaN 2013-09-04 01:00:00  NaN
2  NaN  NaN 2013-09-04 02:00:00  NaN
3  NaN  NaN 2013-09-04 03:00:00  NaN
4  NaN  NaN 2013-09-04 04:00:00  NaN
5  NaN  NaN 2013-09-04 05:00:00  NaN
6  NaN  NaN 2013-09-04 06:00:00  NaN
7  NaN  NaN 2013-09-04 07:00:00  NaN
8  NaN  NaN 2013-09-04 08:00:00  NaN
9  NaN  NaN 2013-09-04 09:00:00  NaN


In [19]:
num_rows = len(hourly.)
for index, row in df.iterrows():
    print(index)
    print(row)
    break

0
lat                           55.9451
lon                          -4.74678
time        2013-09-04 16:19:01+00:00
time_dif                            0
dis_dif                             0
Name: 0, dtype: object
